In [3]:
import torch
import joblib

In [5]:
data = joblib.load('/home/mcarroll/Documents/cd-2/humanoid_tracking/whole_body_tracking/logs/rsl_rl/g1_flat/2025-08-26_10-49-04_lefan_walk/collection/v2/0.0/robot_sa_dataset_r5.pkl')

In [8]:
motion = data[0]['motion']

In [11]:
import sys
sys.path.insert(0, '/home/mcarroll/Documents/cd-2/humanoid_tracking/whole_body_tracking/MDM')
from data_loaders.dataset import extract_observation_terms

In [24]:
def create_causal_attention_mask(sequence_type_array):
    """
    Creates a custom attention mask for a sequence of actions and states.

    Args:
        sequence_type_array (list or torch.Tensor): A boolean array where 
                                                     True indicates a state and False indicates an action.
    
    Returns:
        torch.Tensor: The custom attention mask of shape (seq_len, seq_len).


    A0 [[ True, False, False, False, False, False],
    S0  [ True,  True, False, False, False, False],
    A1  [ True,  True,  True, False, False, False],
    S1  [ True,  True,  True,  True, False, False],
    A2  [ True,  True,  True,  True,  True, False],
    S2  [ True,  True,  True,  True,  True,  True]]
          A0     S0     A1     S1     A2     S2  
    """
    seq_len = len(sequence_type_array)
    is_state = torch.tensor(sequence_type_array, dtype=torch.bool)
    
    # Start with a standard causal mask (tril) to prevent attending to future elements
    mask = torch.tril(torch.ones(seq_len, seq_len, dtype=torch.bool))



    # Apply the custom rules
    for i in range(seq_len):
        if is_state[i]:
            # If the current token is a state, it can attend to all other states.
            # This logic needs to override the causal mask for other states.
            # A state at index i can attend to any state j, regardless of j's position.
            state_indices = torch.where(is_state)[0]
            action_indices = torch.where(~is_state)[0]
            mask[i, state_indices] = True
            mask[i, action_indices] = False
        else:
            # If the current token is an action, it can only attend to
            # past actions and past/current states (this is handled by the tril mask).
            pass # The initial causal mask is sufficient here.

    return ~mask


In [25]:
create_causal_attention_mask([False, True, False, True, False, True])

tensor([[False,  True,  True,  True,  True,  True],
        [ True, False,  True, False,  True, False],
        [False, False, False,  True,  True,  True],
        [ True, False,  True, False,  True, False],
        [False, False, False, False, False,  True],
        [ True, False,  True, False,  True, False]])